
# 🧠 Bayesian Cyber Risk Simulation

This notebook provides a fully-documented, production-grade workflow for cyber loss quantification using a **Bayesian frequency/severity model** combined with a **Markov-chain attacker progression** through **MITRE ATT&CK** stages and **FAIR** loss taxonomy. It is designed for both analytical rigor and stakeholder communication.

### What you will get
- **Posterior inference** for frequency (λ) and per-stage success probabilities
- **Attacker behavior simulation** with retries, fallbacks, detection, and furthest-stage tracking
- **FAIR-aligned losses** (lognormal bodies + Pareto tails for catastrophic events)
- **Option B** support for observed incident counts (Poisson likelihood)
- **Long-tail-aware visualizations** and **CSV exports** for downstream reporting

> **Tip:** This is configured with a larger sampling sizes and may take a while. You can adjust settings in the next section if needed.



## 1️⃣ Configuration and Imports (Production Defaults)

This block imports all required libraries and defines global configuration. Key elements:
- **Sampling settings** for PyMC (draws/tune/chains) are set for production-level fidelity.
- **Monte Carlo** settings determine how many attacker attempts are simulated per posterior draw.
- **Markov parameters** control retries, fallbacks, and detection dynamics per stage.
- **Observed data** (`observed_total_incidents`, `observed_years`) are included but default to `None` so the model runs prior-driven until you populate them.
- **Visualization** preference for plotting in USD or millions.

If running in environments where multiprocessing is constrained (e.g., some Jupyter setups), keep `RUN_PARALLEL=False`.


In [1]:

import os, random, datetime, multiprocessing
from concurrent.futures import ProcessPoolExecutor
from typing import Dict
import numpy as np, pandas as pd, pymc as pm
from scipy import stats, optimize
import matplotlib.pyplot as plt, seaborn as sns

%matplotlib inline
sns.set_style('whitegrid')

# Frequency prior (attempts/year), elicited as a 90% CI -> Lognormal params
CI_MIN_FREQ = 1
CI_MAX_FREQ = 30
Z_90 = 1.645

# PyMC sampling (tune for speed vs. accuracy)
N_SAMPLES = 1500
N_TUNE = 1500
N_CHAINS = 4
TARGET_ACCEPT = 0.90 
RANDOM_SEED = 42

# ---- Monte Carlo attempts per posterior draw ----
RUN_PARALLEL = False       # Set True only if your notebook environment supports multiprocessing
N_WORKERS = None           # None -> use cpu_count()
N_SIM_PER_DRAW = 500       # Per-draw attempts to estimate per-attempt success

# ---- Markov behavior for attacker progression ----
MAX_RETRIES_PER_STAGE = 3
RETRY_PENALTY = 0.75       # geometric penalty to success prob on each retry
FALLBACK_PROB = 0.25       # chance to back up and try previous stage after exhausting retries
DETECT_BASE = 0.01         # baseline chance of detection on a failed try
DETECT_INC_PER_RETRY = 0.06# detection increase with each retry

# ---- Visualization ----
PLOT_IN_MILLIONS = True

# ---- Option B: observed incident counts over an exposure window (optional) ----
# Set both integers to condition posterior; leave as None to skip.
observed_total_incidents = None   # e.g., 7 if 7 successful incidents observed
observed_years = None             # e.g., 3 if those occurred over 3 years



## 2️⃣ Prior Construction: MITRE ATT&CK Stage Success and FAIR Loss Bodies/Tails

**Goal:** Convert SME/control-effectiveness uncertainty into usable statistical priors.

- We map each **MITRE ATT&CK** stage to a **control-effectiveness 90% CI**. Since the model needs attacker *success* probabilities per stage, we invert those ranges and fit **Beta(α,β)** using quantile matching (5% and 95% points).
- For **FAIR** categories, we parameterize **lognormal** bodies from 5%/95% quantiles and add **Pareto** tails (for rare catastrophic events) for **Regulatory/Legal** and **Reputation/Competitive**.

> This separation lets you later plug in organization-specific priors without changing the rest of the pipeline.


In [ ]:

# SME-provided 90% CI for control effectiveness per MITRE stage.
# We convert to attacker success: success ∈ [1 - hi, 1 - lo], then fit Beta via quantile-matching.
STAGE_CONTROL_MAP = {
    "Initial Access":        (0.15, 0.50),
    "Execution":             (0.05, 0.30),
    "Persistence":           (0.10, 0.45),
    "Privilege Escalation":  (0.10, 0.35),
    "Defense Evasion":       (0.20, 0.60),
    "Credential Access":     (0.15, 0.50),
    "Discovery":             (0.02, 0.20),
    "Lateral Movement":      (0.25, 0.65),
    "Collection":            (0.05, 0.30),
    "Command and Control":   (0.10, 0.45),
    "Exfiltration":          (0.30, 0.70),
    "Impact":                (0.35, 0.75),
}
MITRE_STAGES = list(STAGE_CONTROL_MAP.keys())

def _quantile_match_beta(p5, p95, q_low=0.05, q_high=0.95):
    mean = 0.5*(p5+p95)
    width = max(p95-p5, 1e-6)
    conc_guess = 20.0 * (0.1/width)
    a0 = max(1e-3, mean*conc_guess)
    b0 = max(1e-3, (1-mean)*conc_guess)
    def resid(params):
        a,b = params
        if a<=0 or b<=0: return [1e6,1e6]
        return [stats.beta.ppf(q_low,a,b)-p5, stats.beta.ppf(q_high,a,b)-p95]
    sol = optimize.root(resid, [a0,b0], method="hybr")
    if sol.success and np.all(np.array(sol.x) > 0): return float(sol.x[0]), float(sol.x[1])
    return a0,b0

_success_90s = [(1.0 - hi, 1.0 - lo) for lo, hi in STAGE_CONTROL_MAP.values()]
alphas, betas = zip(*(_quantile_match_beta(lo,hi) for lo,hi in _success_90s))
alphas, betas = np.array(alphas), np.array(betas)

# --- FAIR taxonomy (bodies via lognormal, tails via Pareto) ---
loss_categories = ["Productivity","ResponseContainment","RegulatoryLegal","ReputationCompetitive"]
loss_q5_q95 = {
    "Productivity": (10_000, 150_000),
    "ResponseContainment": (20_000, 500_000),
    "RegulatoryLegal": (0, 1_000_000),
    "ReputationCompetitive": (0, 2_000_000),
}
def _lognormal_from_q5_q95(q5,q95):
    q5, q95 = max(q5,1.0), max(q95, q5*1.0001)
    ln5, ln95 = np.log(q5), np.log(q95)
    sigma = (ln95 - ln5) / (2.0 * Z_90)
    mu = 0.5 * (ln5 + ln95)
    return mu, sigma
cat_mu = np.zeros(len(loss_categories)); cat_sigma = np.zeros(len(loss_categories))
for i,cat in enumerate(loss_categories):
    cat_mu[i], cat_sigma[i] = _lognormal_from_q5_q95(*loss_q5_q95[cat])
pareto_defaults = {
    "RegulatoryLegal": {"xm": 100_000.0, "alpha": 1.8},
    "ReputationCompetitive": {"xm": 250_000.0, "alpha": 1.5}
}
print("MITRE priors and FAIR loss parameters configured.")



## 3️⃣ Bayesian Model & Sampling (λ and Stage Success)

This section builds and samples a **PyMC** model that estimates:
- **λ** (annual attack attempt frequency), modeled as a **Lognormal** with a 90% CI prior.
- **Stage success probabilities** across MITRE ATT&CK, modeled as independent **Betas** derived above.

It also adds an **optional observed-data likelihood** (Option B):
- If you provide `observed_total_incidents` and `observed_years`, the model includes `Poisson(λ × years)` and conditions on that evidence.
- If either is `None`, the model runs **prior-only**.

We then sample using **NUTS** with production-level settings defined earlier.


In [ ]:

with pm.Model() as model:
    mu_lambda = np.log(np.sqrt(CI_MIN_FREQ * CI_MAX_FREQ))
    sigma_lambda = (np.log(CI_MAX_FREQ) - np.log(CI_MIN_FREQ)) / (2.0 * Z_90)
    lambda_rate = pm.Lognormal("lambda_rate", mu=mu_lambda, sigma=sigma_lambda)

    success_probs = pm.Beta("success_probs", alpha=alphas, beta=betas, shape=len(MITRE_STAGES))

    # Option B: Observed incidents over exposure window
    if observed_total_incidents is not None and observed_years is not None:
        pm.Poisson("obs_incidents", mu=lambda_rate * observed_years, observed=observed_total_incidents)
        print(f"Conditioning on observed data: {observed_total_incidents} incidents over {observed_years} years.")
    else:
        print("No observed incident data provided — running prior-driven.")

    trace = pm.sample(draws=N_SAMPLES, tune=N_TUNE, chains=N_CHAINS,
                      target_accept=TARGET_ACCEPT, random_seed=RANDOM_SEED, progressbar=True)

posterior = trace.posterior
lambda_draws = posterior["lambda_rate"].values.reshape(-1)
success_probs_draws = posterior["success_probs"].values.reshape(-1, len(MITRE_STAGES))
R = len(lambda_draws)
print(f"Posterior draws: {R}")



## 4️⃣ Markov Attacker Simulation with Progression Recording

This block simulates attacker behavior per posterior draw, following the MITRE ATT&CK sequence. Each attempt can:
- **Retry** a stage up to a maximum, with **geometric penalty** to success per retry.
- **Trigger detection** on failed tries, with probability growing per retry.
- **Fallback** one stage (with a specified probability) after exhausting retries.
- Either **fail early** or **complete** all stages (full compromise).

We record per-attempt details and **aggregate** across attempts:
- Attempts and retries **per stage**
- Distribution of **furthest stage reached**
- Total **fallbacks** and **detections**
- Estimated **per-attempt success rate**

> In notebooks, we default to **serial** execution for reliability; you can flip to parallel if supported.


In [ ]:

from typing import Dict

def simulate_one_attempt_record(success_probs_stage,
                                rng: random.Random,
                                max_retries_per_stage=MAX_RETRIES_PER_STAGE,
                                retry_penalty=RETRY_PENALTY,
                                fallback_prob=FALLBACK_PROB,
                                detect_base=DETECT_BASE,
                                detect_increase_per_retry=DETECT_INC_PER_RETRY) -> Dict:
    nstages = len(success_probs_stage)
    stage_idx = 0
    retries_by_stage = [0] * nstages
    stages_tried = []
    fallback_count = 0
    detected = False
    furthest = 0
    while True:
        if stage_idx >= nstages:
            furthest = nstages
            return {"success": True, "stages_tried": stages_tried, "retries_by_stage": retries_by_stage,
                    "fallback_count": fallback_count, "detected": detected, "furthest_stage": furthest}
        p_base = float(success_probs_stage[stage_idx])
        progressed = False
        for k in range(max_retries_per_stage + 1):
            stages_tried.append(stage_idx)
            p_try = (retry_penalty ** k) * p_base
            if rng.random() < p_try:
                retries_by_stage[stage_idx] += k
                progressed = True
                break
            detect_prob = min(max(detect_base + detect_increase_per_retry * k, 0.0), 1.0)
            if rng.random() < detect_prob:
                detected = True
                furthest = max(furthest, stage_idx)
                return {"success": False, "stages_tried": stages_tried, "retries_by_stage": retries_by_stage,
                        "fallback_count": fallback_count, "detected": detected, "furthest_stage": furthest}
        if not progressed:
            if stage_idx == 0 or rng.random() > fallback_prob:
                furthest = max(furthest, stage_idx)
                return {"success": False, "stages_tried": stages_tried, "retries_by_stage": retries_by_stage,
                        "fallback_count": fallback_count, "detected": detected, "furthest_stage": furthest}
            fallback_count += 1
            stage_idx -= 1
            continue
        furthest = max(furthest, stage_idx + 1)
        stage_idx += 1

def worker_function_record(args):
    per_stage, n_sim, seed = args
    rng = random.Random(int(seed))
    nstages = len(per_stage)
    stage_attempts = np.zeros(nstages, dtype=int)
    stage_retries = np.zeros(nstages, dtype=int)
    furthest_counts = np.zeros(nstages + 1, dtype=int)
    successes = 0
    total_fallbacks = 0
    total_detections = 0
    for _ in range(int(n_sim)):
        rec = simulate_one_attempt_record(per_stage, rng)
        for s in rec["stages_tried"]:
            stage_attempts[int(s)] += 1
        for i,r in enumerate(rec["retries_by_stage"]):
            stage_retries[i] += int(r)
        furthest_counts[int(rec["furthest_stage"])] += 1
        successes += int(rec["success"])
        total_fallbacks += int(rec["fallback_count"])
        total_detections += int(rec["detected"])
    return {"n_sim": int(n_sim), "successes": int(successes),
            "stage_attempts": stage_attempts, "stage_retries": stage_retries,
            "furthest_counts": furthest_counts, "fallbacks": int(total_fallbacks),
            "detections": int(total_detections)}

# Per-draw per-attempt success estimation
p_success_simulated = np.zeros(R, dtype=float)
args_list = [(success_probs_draws[i, :], N_SIM_PER_DRAW, 1000 + i) for i in range(R)]

total_stage_attempts = np.zeros(len(MITRE_STAGES), dtype=int)
total_stage_retries = np.zeros(len(MITRE_STAGES), dtype=int)
total_furthest = np.zeros(len(MITRE_STAGES) + 1, dtype=int)
total_successes = 0
total_simulated = 0
total_fallbacks = 0
total_detections = 0

if RUN_PARALLEL:
    with ProcessPoolExecutor(max_workers=(N_WORKERS or multiprocessing.cpu_count())) as exe:
        for i, res in enumerate(exe.map(worker_function_record, args_list)):
            p_success_simulated[i] = res["successes"] / float(res["n_sim"])
            total_simulated += res["n_sim"]; total_successes += res["successes"]
            total_stage_attempts += res["stage_attempts"]; total_stage_retries += res["stage_retries"]
            total_furthest += res["furthest_counts"]; total_fallbacks += res["fallbacks"]
            total_detections += res["detections"]
else:
    for i, args in enumerate(args_list):
        res = worker_function_record(args)
        p_success_simulated[i] = res["successes"] / float(res["n_sim"])
        total_simulated += res["n_sim"]; total_successes += res["successes"]
        total_stage_attempts += res["stage_attempts"]; total_stage_retries += res["stage_retries"]
        total_furthest += res["furthest_counts"]; total_fallbacks += res["fallbacks"]
        total_detections += res["detections"]

print("Per-draw attacker simulations complete.")



## 5️⃣ Posterior Predictive Loss Simulation (FAIR)

For each posterior draw we simulate **annual losses**:
- Draw a successful incident count from **Poisson(λ × p_success)**.
- For each incident, sample **severity multipliers** that scale lognormal FAIR category bodies.
- For **Regulatory/Legal** and **Reputation/Competitive**, add **zero-inflated Pareto tails** to capture rare, large-impact events.

This produces arrays of annual total losses and per-category losses for summary and visualization.


In [ ]:

rng_np = np.random.default_rng(RANDOM_SEED + 1)
annual_losses = np.zeros(R, dtype=float)
incident_counts = np.zeros(R, dtype=int)
cat_loss_matrix = np.zeros((R, len(loss_categories)), dtype=float)

for i in range(R):
    lam_eff = lambda_draws[i] * p_success_simulated[i]
    n_succ = rng_np.poisson(lam_eff)
    incident_counts[i] = n_succ
    if n_succ == 0:
        continue

    # Incident-level severity multipliers
    severities = rng_np.lognormal(mean=0.0, sigma=0.6, size=n_succ)

    # Primary categories (always-on)
    prod = np.sum(rng_np.lognormal(cat_mu[0], cat_sigma[0], size=n_succ) * severities)
    resp = np.sum(rng_np.lognormal(cat_mu[1], cat_sigma[1], size=n_succ) * severities)

    # --- Per-incident correlated Pareto tails ---
    tail_trigger_prob = 0.10
    reg = 0.0
    rep = 0.0
    for s in range(n_succ):
        sev = severities[s]

        # Regulatory/Legal tail
        if rng_np.random() < tail_trigger_prob:
            xm, alpha = pareto_defaults["RegulatoryLegal"]["xm"], pareto_defaults["RegulatoryLegal"]["alpha"]
            reg += (xm * (1.0 + rng_np.pareto(alpha))) * (1.0 + sev)**1.2

        # Reputation/Competitive tail
        if rng_np.random() < tail_trigger_prob:
            xm, alpha = pareto_defaults["ReputationCompetitive"]["xm"], pareto_defaults["ReputationCompetitive"]["alpha"]
            rep += (xm * (1.0 + rng_np.pareto(alpha))) * (1.0 + sev)**1.2

    # Aggregate
    cat_loss_matrix[i, :] = [prod, resp, reg, rep]
    annual_losses[i] = prod + resp + reg + rep

print("Posterior predictive losses computed.")



## 6️⃣ Summary Metrics & Progression Analysis

We compile high-level KPIs and attacker behavior summaries:
- **AAL** mean/median and **95% credible interval** for total annual loss
- **Category-level** credible intervals and **share of median AAL**
- **Attacker progression** metrics: per-attempt success, fallbacks per attempt, detection rate
- Intermediate arrays maintained for downstream plots and CSV export


In [ ]:

mean_AAL = annual_losses.mean()
median_AAL = np.median(annual_losses)
p2_5, p97_5 = np.percentile(annual_losses, [2.5, 97.5])
mean_incidents = incident_counts.mean()
pct_zero = 100.0 * np.mean(annual_losses == 0.0)

print("\nAAL posterior predictive summary:")
print(f"Mean AAL: ${mean_AAL:,.0f}")
print(f"Median AAL: ${median_AAL:,.0f}")
print(f"AAL 95% credible interval (annualized total loss): ${p2_5:,.0f} – ${p97_5:,.0f}")
print(f"Mean successful incidents / year: {mean_incidents:.2f}")
print(f"% years with zero successful incidents: {pct_zero:.1f}%\n")

print("Category-level annual loss 95% credible intervals:")
for c, cat in enumerate(loss_categories):
    low, med, high = np.percentile(cat_loss_matrix[:, c], [2.5, 50, 97.5])
    share_med = 100.0 * med / (median_AAL + 1e-12)
    print(f"  {cat:<24s} ${low:,.0f} – ${high:,.0f}  (median ${med:,.0f}, ≈{share_med:.1f}% of median AAL)")

# Attacker progression aggregates
overall_success_rate = total_successes / total_simulated
attempts_per_stage = total_stage_attempts / total_simulated
avg_retries_per_stage = total_stage_retries / (total_stage_attempts + 1e-12)
furthest_stage_prob = total_furthest / total_simulated
fallbacks_per_attempt = total_fallbacks / total_simulated
detection_rate = total_detections / total_simulated

print("\nAttacker progression summary:")
print(f"Overall per-attempt success: {overall_success_rate:.4f}")
print(f"Avg fallbacks per attempt: {fallbacks_per_attempt:.4f}")
print(f"Detection rate: {detection_rate:.4f}")



## 7️⃣ Visualization — Long-tail-aware 2×2 Dashboard + Log-scale Tail

This creates a **2×2 dashboard** showing:
1. Posterior of **λ (attacks/year)**
2. Posterior of **per-attempt success probability**
3. Posterior predictive **successful incidents/year**
4. Posterior predictive **annual loss** (clipped to central mass for readability)

Then, we add a **log-scale tail plot** for annual loss to inspect heavy tails.  
Percentile lines (P50/P90/P95/P99) are drawn to anchor interpretation.


In [ ]:

def auto_clip(data, low=0.001, high=0.999):
    if len(data) == 0: return data
    low_v, high_v = np.percentile(data, [low*100, high*100])
    return data[(data >= low_v) & (data <= high_v)]

def annotate_percentiles(ax, data, percentiles=(50,90,95,99), scale=1.0, money=True):
    if len(data) == 0: return
    ymax = ax.get_ylim()[1] if ax.get_ylim()[1] > 0 else 1.0
    for p in percentiles:
        val = np.percentile(data, p)/scale
        ax.axvline(val, color='k', linestyle='--', lw=0.8, alpha=0.85)
        label = f"P{p}=" + (f"${val:,.0f}" if money else f"{val:,.3f}")
        ax.text(val, ymax*0.92, label, rotation=90, va='top', ha='center', fontsize=8, backgroundcolor='white')

scale = 1e6 if PLOT_IN_MILLIONS else 1.0
scale_label = "Million USD" if PLOT_IN_MILLIONS else "USD"

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# (1) Posterior λ
data_lambda = auto_clip(lambda_draws)
axes[0, 0].hist(data_lambda, bins=40, color='steelblue', alpha=0.85)
axes[0, 0].set_title("Posterior λ (attacks/year)")
axes[0, 0].set_xlabel("λ (attacks/year)"); axes[0, 0].set_ylabel("Frequency")
annotate_percentiles(axes[0, 0], data_lambda, money=False)

# (2) Per-attempt success probability (simulated)
data_success = auto_clip(p_success_simulated)
axes[0, 1].hist(data_success, bins=40, color='steelblue', alpha=0.85)
axes[0, 1].set_title("Per-attempt success probability (simulated)")
axes[0, 1].set_xlabel("Probability"); axes[0, 1].set_ylabel("Frequency")
annotate_percentiles(axes[0, 1], data_success, money=False)

# (3) Successful incidents/year
data_incidents = auto_clip(incident_counts)
axes[1, 0].hist(data_incidents, bins=40, color='steelblue', alpha=0.85)
axes[1, 0].set_title("Successful incidents / year (posterior predictive)")
axes[1, 0].set_xlabel("Count"); axes[1, 0].set_ylabel("Frequency")
annotate_percentiles(axes[1, 0], data_incidents, money=False)

# (4) Annual loss (USD)
nonzero = annual_losses[annual_losses > 0]
if len(nonzero) > 0:
    data_loss = auto_clip(nonzero)
    axes[1, 1].hist(data_loss / scale, bins=80, color='steelblue', alpha=0.85)
    axes[1, 1].set_title("Annual Loss (posterior predictive)")
    axes[1, 1].set_xlabel(f"Annual Loss ({scale_label})"); axes[1, 1].set_ylabel("Frequency")
    annotate_percentiles(axes[1, 1], data_loss, scale=scale, money=True)
else:
    axes[1, 1].text(0.5, 0.5, "All draws are zero", ha="center", va="center")

plt.tight_layout(); plt.show()

# Log-scale heavy-tail view
if len(nonzero) > 0:
    low_p, high_p = np.percentile(nonzero, [0.1, 99.9])
    filtered = nonzero[(nonzero >= low_p) & (nonzero <= high_p)]
    filtered = filtered if len(filtered) >= 10 else nonzero
    bins = np.logspace(np.log10(filtered.min()/scale), np.log10(filtered.max()/scale), 100)
    plt.figure(figsize=(8,5))
    plt.hist(filtered/scale, bins=bins, color='tomato', alpha=0.8)
    plt.xscale('log')
    plt.title('Annual Loss — Log scale (central body shown)')
    plt.xlabel(f'Annual Loss ({scale_label})'); plt.ylabel('Frequency')
    annotate_percentiles(plt.gca(), filtered, scale=scale, money=True)
    plt.grid(True, which='both', ls='--', alpha=0.35)
    plt.tight_layout(); plt.show()



## 8️⃣ Attacker Progression Visualization

We translate the aggregated attacker metrics into interpretable charts:
- **Stage reach probability** (chance an attacker got to at least each stage)
- **Attempts per stage** (log-scaled to accommodate high variance)
- **Average retries per attempt** (conditional on a stage being attempted)
- **Furthest-stage distribution**, with the last bar labeled **Full Compromise**

These views help identify the **choke points** and stages where **defenses or detections** are most effective.


In [ ]:

sns.set_style("whitegrid")
nstages = len(MITRE_STAGES)

# Probability attacker reached at least each stage
furthest_stage_prob = total_furthest / total_simulated
reached_at_least = np.array([np.sum(furthest_stage_prob[k+1:]) for k in range(nstages)])

plt.figure(figsize=(12, 3))
sns.barplot(x=MITRE_STAGES, y=reached_at_least)
plt.xticks(rotation=45, ha='right'); plt.ylabel('Probability reached ≥ stage')
plt.title('Stage reach probability'); plt.tight_layout(); plt.show()

# Attempts per stage (log scale)
attempts_per_stage = total_stage_attempts / total_simulated
plt.figure(figsize=(12, 3)); sns.barplot(x=MITRE_STAGES, y=attempts_per_stage)
plt.yscale('log'); plt.xticks(rotation=45, ha='right'); plt.ylabel('Avg attempts per simulated attempt (log)')
plt.title('Attempts per stage'); plt.tight_layout(); plt.show()

# Avg retries per attempt (conditional on attempt)
avg_retries_per_stage = total_stage_retries / (total_stage_attempts + 1e-12)
plt.figure(figsize=(12, 3)); sns.barplot(x=MITRE_STAGES, y=avg_retries_per_stage)
plt.xticks(rotation=45, ha='right'); plt.ylabel('Avg retries per attempt'); plt.title('Retries per stage')
plt.tight_layout(); plt.show()

# Furthest stage distribution
labels = [f"Reached_{i}" for i in range(nstages)] + ["Full Compromise"]
plt.figure(figsize=(12, 3)); sns.barplot(x=labels, y=furthest_stage_prob)
plt.xticks(rotation=45, ha='right'); plt.ylabel('Probability'); plt.title('Furthest stage reached')
plt.tight_layout(); plt.show()



## 9️⃣ CSV Export for Reuse / Reporting

We export three CSVs in the notebook's working directory with a timestamp suffix:
1. **Detailed results** — one row per posterior draw, with λ, per-attempt success, incidents, total annual loss, and per-category losses.
2. **Summary statistics** — mean/median/95% CI by category, plus share of median AAL.
3. **Attacker progression stats** — stage reach probabilities, attempts per stage, and retries per attempt.

These files can be pulled into BI tools, spreadsheets, or versioned for audit.


In [ ]:

script_dir = os.getcwd()
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Detailed per-draw results
df = pd.DataFrame({
    "lambda_draw": lambda_draws,
    "p_success": p_success_simulated,
    "incidents": incident_counts,
    "annual_loss": annual_losses,
    **{cat: cat_loss_matrix[:, i] for i, cat in enumerate(loss_categories)}
})
res_name = os.path.join(script_dir, f"cyber_risk_simulation_results_{timestamp}.csv")
df.to_csv(res_name, index=False)

# Summary stats
summary_rows = [{
    "Category": "Total Annual Loss",
    "Mean": float(np.mean(annual_losses)),
    "Median": float(np.median(annual_losses)),
    "CI_2.5%": float(np.percentile(annual_losses, 2.5)),
    "CI_97.5%": float(np.percentile(annual_losses, 97.5)),
    "Median Share of AAL (%)": 100.0
}]
median_AAL_val = float(np.median(annual_losses))
for i, cat in enumerate(loss_categories):
    vals = cat_loss_matrix[:, i]
    summary_rows.append({
        "Category": cat,
        "Mean": float(np.mean(vals)),
        "Median": float(np.median(vals)),
        "CI_2.5%": float(np.percentile(vals, 2.5)),
        "CI_97.5%": float(np.percentile(vals, 97.5)),
        "Median Share of AAL (%)": float(100.0 * (np.median(vals) / (median_AAL_val + 1e-12)))
    })
summary_df = pd.DataFrame(summary_rows)
sum_name = os.path.join(script_dir, f"cyber_risk_simulation_summary_{timestamp}.csv")
summary_df.to_csv(sum_name, index=False)

# Attacker progression stats
attacker_rows = []
for i, stage in enumerate(MITRE_STAGES):
    attacker_rows.append({
        "Stage": stage,
        "Reach_Prob": float(reached_at_least[i]),
        "Avg_Attempts_per_sim": float(attempts_per_stage[i]),
        "Avg_Retries_per_attempt": float(avg_retries_per_stage[i])
    })
attacker_df = pd.DataFrame(attacker_rows)
attacker_name = os.path.join(script_dir, f"cyber_risk_simulation_attacker_stats_{timestamp}.csv")
attacker_df.to_csv(attacker_name, index=False)

print("CSV exports written to:", script_dir)
print(" -", res_name)
print(" -", sum_name)
print(" -", attacker_name)



© 2025 — FAIR–MITRE ATT&CK Quantitative Cyber Risk Framework

Author: Joshua M. Connors

License: Open Use for Non-Commercial Research and Risk Analysis